Pandahub comes with an application level user management. All necessary data is stored in the collection "user_management". If pandahub is used via its REST interface, this allows for user authentification via https.

***However***, you can also use the pandahub API ***without*** running the REST interface. Instead, you can use it just as an convenient API to store network data into a mongo DB.

If you still need some control over the access to the data you must fall back on the mongoDB user access management. In the following is a short introduction how this can be accomplished.

The following is only necessary if you do not rely on the pandahub application level user management!

In [ ]:
MONGODB_IP = "127.0.0.1"
MONGODB_PORT = 27017
# we need a user that has the role "userAdminAnyDatabase" to be able to see current users and create new users
MONGODB_ROOT_USER = "root"

In [ ]:
import getpass
import pandahub
print(f"insert password for user {MONGODB_ROOT_USER}")
MONGODB_URL = f"mongodb://{MONGODB_ROOT_USER}:{getpass.getpass()}@{MONGODB_IP}:{MONGODB_PORT}"
ph = pandahub.PandaHub(connection_url=MONGODB_URL)

In order to list existing MongoDB users and theirs respective access rights to the different project databases:

In [ ]:
import pandas as pd
userlist = ph.mongo_client["admin"]["system.users"].find({}, projection={"user": 1, "roles": 1})
users = pd.DataFrame([(l["user"], r["db"], r["role"]) for l in userlist for r in l["roles"]], columns=["user", "db", "role"])
users[~users.db.isin(["admin", "user_management"])]  # we can exclude some databases for better clarity

We can create new users for existing projects:

In [ ]:
user = "<username>"
pwd = "<password>"
# create a user that has read access to 'user_management' of pandahub
ph.mongo_client.admin.command('createUser', user, pwd=pwd, roles=[{'role': 'read', 'db': 'user_management'}])

Now we can grant readWrite access to the project database(s).

In [ ]:
project_id = "<project_id>"
ph.mongo_client.admin.command('grantRolesToUser', user, roles=[{'role': 'readWrite', 'db': project_id}])

Remove users entirely via

In [ ]:
user = "<username>"
ph.mongo_client.admin.command("dropUser", user)